# Split datasets


In [1]:
import json
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

In [23]:
# label_map_csv = "../datasets/cassava-leaf-disease-classification/train.csv"
# label_map_csv = "../datasets/PAD-UFES-20/metadata.csv"
label_map_csv = "../datasets/ddi-diverse-dermatology-images/ddi_metadata.csv"
images_root = "../datasets/ddi-diverse-dermatology-images/"
split_column = "malignant"
file_column = "DDI_file"
image_type = ""

label_map_csv = "../datasets/HAM10000/HAM10000_metadata"
images_root = "../datasets/HAM10000/images/"
split_column = "dx"
file_column = "image_id"
image_type = ".jpg"

label_map_csv = "../datasets/PAD-UFES-20/metadata.csv"
images_root = "../datasets/PAD-UFES-20/images/"
split_column = "diagnostic"
file_column = "img_id"
image_type = ""

label_map_csv = "../datasets/ddi-diverse-dermatology-images/ddi_metadata.csv"
images_root = "../datasets/ddi-diverse-dermatology-images/"
split_column = "disease"
file_column = "DDI_file"
image_type = ""


In [24]:
df_full = pd.read_csv(label_map_csv)
df_full.columns

Index(['Unnamed: 0', 'DDI_ID', 'DDI_file', 'skin_tone', 'malignant',
       'disease'],
      dtype='object')

In [25]:
for value in df_full[split_column].unique():
    subdirectory = os.path.join(images_root,split_column,str(value))
    print(f"Copying to {subdirectory}")
    os.makedirs(subdirectory, exist_ok=True)
    for image_name in df_full[df_full[split_column] == value][file_column]:
        image_name += image_type
        shutil.copy(os.path.join(images_root,image_name), f"{subdirectory}/")

Copying to ../datasets/ddi-diverse-dermatology-images/disease\melanoma-in-situ
Copying to ../datasets/ddi-diverse-dermatology-images/disease\mycosis-fungoides
Copying to ../datasets/ddi-diverse-dermatology-images/disease\squamous-cell-carcinoma-in-situ
Copying to ../datasets/ddi-diverse-dermatology-images/disease\basal-cell-carcinoma
Copying to ../datasets/ddi-diverse-dermatology-images/disease\squamous-cell-carcinoma
Copying to ../datasets/ddi-diverse-dermatology-images/disease\melanoma-acral-lentiginous
Copying to ../datasets/ddi-diverse-dermatology-images/disease\basal-cell-carcinoma-superficial
Copying to ../datasets/ddi-diverse-dermatology-images/disease\squamous-cell-carcinoma-keratoacanthoma
Copying to ../datasets/ddi-diverse-dermatology-images/disease\subcutaneous-t-cell-lymphoma
Copying to ../datasets/ddi-diverse-dermatology-images/disease\melanocytic-nevi
Copying to ../datasets/ddi-diverse-dermatology-images/disease\seborrheic-keratosis-irritated
Copying to ../datasets/ddi-di

In [26]:
# df_train, df_test = train_test_split(
#     df_full, test_size=test_ratio, stratify=df_full["label"], random_state=seed
# )
# print(df_train["label"].value_counts())
# print(df_test["label"].value_counts())

# _, df_train_mini = train_test_split(
#     df_train, test_size=0.1, stratify=df_train["label"], random_state=seed
# )
# print(df_train_mini["label"].value_counts())

In [27]:
# entries = json.load(
#     open(
#         "../datasets/cassava-leaf-disease-classification/label_num_to_disease_map.json"
#     )
# )
# entries

In [28]:
# for key in entries:
#     value = entries[key]
#     os.makedirs(f"../datasets/cassava-dataset/train/{value}/", exist_ok=True)
#     os.makedirs(f"../datasets/cassava-dataset/test/{value}/", exist_ok=True)
#     os.makedirs(f"../datasets/cassava-dataset/train-mini/{value}/", exist_ok=True)

In [29]:
# for index, row in df_test.iterrows():
#     image_id = row["image_id"]
#     label_key = row["label"]
#     shutil.copy(
#         f"../datasets/cassava-leaf-disease-classification/train_images/{image_id}", 
#         f"../datasets/cassava-dataset/test/{entries[str(label_key)]}/"
#     )

In [30]:
# for index, row in df_train.iterrows():
#     image_id = row["image_id"]
#     label_key = row["label"]
#     shutil.copy(
#         f"../datasets/cassava-leaf-disease-classification/train_images/{image_id}", 
#         f"../datasets/cassava-dataset/train/{entries[str(label_key)]}/"
#     )

In [31]:
# for index, row in df_train_mini.iterrows():
#     image_id = row["image_id"]
#     label_key = row["label"]
#     shutil.copy(
#         f"../datasets/cassava-leaf-disease-classification/train_images/{image_id}", 
#         f"../datasets/cassava-dataset/train-mini/{entries[str(label_key)]}/"
#     )